In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

import pandas as pd

import hist
from hist import Hist

import babar_analysis_tools as bat
import myPIDselector

import os 

from analysis_variables import *


## Reading in files

In [ ]:
#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data/bnv_plambda"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
ak_sp, ak_data = bat.load_datasets(topdir=topdir, subset='Run1')

#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='all')
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')

In [ ]:
'''
start = time.time()

## At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At home (Bellis)
#topdir = '/home/bellis/babar_data/bnv_plambda'

## Josie Laptop
topdir = "/Users/josieswann/Babar_analyses/BNV_pLambda/"

### Read in SP- includes both signal and background 
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

### convert files from a parquet file to an awkward array
start = time.time()
ak_sp = ak.from_parquet(filename)
print(f"Took {time.time() - start} s")


# Read in data
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'
start = time.time()
ak_data = ak.from_parquet(filename)
print(f"Took {time.time() - start} s")
'''

In [ ]:
ak_sp["spmode"]
#np.unique(ak_sp['spmode'])

In [ ]:
### READ IN THE BIG SP FILE--> ALL RUNS! Doesn't work on Luminous 
'''
big_file= f"{topdir}/Background_and_signal_SP_modes_All_runs.parquet"

start= time.time()
ak_all_runs_blinded= ak.from_parquet(big_file)
print(f"Yikes! Took {time.time()-start} s")
''';

In [ ]:
### READ IN THE BIG DATA FILE--> ALL RUNS! 
'''
big_file= f"{topdir}/Data_All_runs_BLINDED.parquet"

start= time.time()
ak_all_runs_blinded= ak.from_parquet(big_file)
print(f"Phew! Took {time.time()-start} s")
''';

In [ ]:
### information about cross section --> what we'll use to calculate scaling values for histograms 

dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

In [ ]:
#bat.table_from_df(no_notes,"shortened_stats")

In [ ]:
# Let's look at one of these awkward arrays!

ak_sp;

In [ ]:
# Awkward arrays can be different "depths"

# This is just 1 level deep because it contains information about each *event*. 
print(ak_sp['spmode'])
print()

# Same with these 
print(ak_sp['nB'])
print()
print(ak_sp['beamSZ'])
print()
print(ak_sp['R2All'])
print()





In [ ]:
# Often in particle physics data, we go another level. 
# For example, if each event has a different number of tracks, 
# we want to be able to organize that 

# This is 2 levels deep 
print(ak_sp['TRKp3'])
print()

# We can look at these closer
for i in range(5):
    print(ak_sp['TRKp3'][i])


# If this seems confusing, that's OK! It is! :)
# But the short of it is
#  * Data where there is one value per event is just 1 level deep
#  * Information about *particles* is 2 (or more) levels deep

In [ ]:
# Why does this matter? 
# Well for our purposes it affects how we use our *masks*
# 
# Do we mask at the event level (e.g. picking out specific SP modes)
# or do we mask at the particle level (only using particles that pass certain criteria)

event_mask = ak_sp['spmode']=='998'
print(len(event_mask))
# Mask at the event level before you create a particle level mask
particle_mask = ak_sp[event_mask]['Lambda0FlightLen'] > 1.0
print(len(particle_mask))

# First mask at the event level
# get your particle data
# mask the particle data
x = ak_sp[event_mask]['Lambda0_unc_Mass'][particle_mask]
x = ak.flatten(x)

plt.hist(x, bins=50);

In [ ]:
# Does this work
event_mask
particle_mask

print(len(particle_mask))
print(len(particle_mask[event_mask]))
print(len(ak_sp[event_mask]['Lambda0_unc_Mass'][particle_mask[event_mask]]))

In [ ]:
print(ak_sp['spmode'])

## Hist Creation and Filling 

In [ ]:
# Testing variables to see their range 

#var= ak_sp["BSphr"]
var= ak.flatten(ak_sp["BLegendreP2"])

plt.hist(var, bins= 100, range= (0,8));

In [ ]:
hist_defs = {}


### Entries followed by lots of hashtags have outliers 
  
hist_defs['BSphr'] =                           {"nbins":100, "lo":0,     "hi":0.2,   "label":"Sphericity"}
hist_defs["BpostFitDeltaE"]=                   {"nbins":100, "lo":-1.0,  "hi":1.0,   "label":"$\Delta$ E"}
hist_defs["BpostFitMes"]=                      {"nbins":100, "lo":5.2,   "hi":5.3,   "label":"$M_{ES}$ [GeV/c$^2$]"}
hist_defs["BThrust"]=                          {"nbins":100, "lo":0.9,   "hi":1.05,  "label":"B thrust"}
hist_defs["BCosThetaS"]=                       {"nbins":150, "lo":-1.05, "hi":1.05,  "label":"B Cos Theta S"}
hist_defs["Lambda0_unc_Mass"]=                 {"nbins":100, "lo":1.105, "hi":1.125, "label":"$\Lambda^0$ mass [GeV/c$^2$]"}
hist_defs["nTracks"]=                          {"nbins":100, "lo":0,     "hi":18,    "label":"number of tracks"}
hist_defs["nGoodTrkLoose"]=                    {"nbins":100, "lo":0,     "hi":14,    "label":"number of Good Tracks- Loose"}
hist_defs["nChargedTracks"]=                   {"nbins":100, "lo":0,     "hi":0.2,   "label":"number of Charged Tracks"}
hist_defs["R2"]=                               {"nbins":100, "lo":0,     "hi":1.05,  "label":"R2"}
hist_defs["R2All"]=                            {"nbins":100, "lo":0,     "hi":1,     "label":"R2All"}
hist_defs["thrustMag"]=                        {"nbins":100, "lo":.6,    "hi":1,     "label":"thrustMag"}
hist_defs["thrustMagAll"]=                     {"nbins":100, "lo":.6,    "hi":1,     "label":"thrustMagAll"}
hist_defs["thrustCosTh"]=                      {"nbins":100, "lo":0,     "hi":1,     "label":"thrustCosTh"}
hist_defs["thrustCosThAll"]=                   {"nbins":100, "lo":0,     "hi":1,     "label":"thrustCosThAll"}
hist_defs["sphericityAll"]=                    {"nbins":100, "lo":0,     "hi":0.75,  "label":"Sphericity"}
hist_defs["BCosSphr"]=                         {"nbins":100, "lo":-.8,   "hi":1,     "label":"BCosSphr"}
hist_defs["BCosThetaT"]=                       {"nbins":100, "lo":-1,    "hi":1,     "label":"BCosThetaT"}
hist_defs["BCosThrust"]=                       {"nbins":100, "lo":0,     "hi":1,     "label":"BCosThrust"}
hist_defs["BLegendreP2"]=                      {"nbins":100, "lo":0,     "hi":7,     "label":"BLegendreP2"}#################
hist_defs["BR2ROE"]=                           {"nbins":100, "lo":0,     "hi":1,     "label":"BR2ROE"}
hist_defs["BSphrROE"]=                         {"nbins":100, "lo":0,     "hi":1,     "label":"BSphrROE"}
hist_defs["BThrustROE"]=                       {"nbins":100, "lo":0.5,   "hi":1,     "label":"BThrustROE"}
hist_defs["Lambda0postFitFlight"]=             {"nbins":100, "lo":-1,    "hi":40,    "label":"Lambda0postFitFlight"} ######################
hist_defs["Lambda0postFitFlightSignificance"]= {"nbins":100, "lo":-20,   "hi":300,   "label":"Lambda0postFitFlightSignificance"} ##################
hist_defs["nTRK"]=                             {"nbins":20, "lo":0,      "hi":20,    "label":"# of charged tracks"}

In [ ]:
def create_empty_histograms(hist_defs): 
    ### Creates empty Hist object histograms based on the information in the dictionary above
    ### Then overwrites the information in the dictionary to be the hist object. All_hists goes from containing unconnected
    ### info about each variable to a single object containing all the same info 
    
    all_hists={}
    for var in hist_defs.keys():
        h = Hist.new.Reg(hist_defs[var]["nbins"], hist_defs[var]["lo"], hist_defs[var]["hi"], name='var', label=f"{hist_defs[var]['label']}") \
                 .StrCat([], name="SP", label="SP modes", growth=True)\
                 .StrCat([], name="cuts", label="Cuts", growth=True)\
                 .Weight()
    
        all_hists[var] = h

    return all_hists


def fill_histograms(ak_arr, empty_hists, spmodes=['998'], weights=[1.0]):
    ### Takes the dictionary of objects we made before and fills them 
    ### with the correct information, based on SP mode and Cut. 
    ### Each cut pares down the background and hopefully makes the signal more apparent
    
    for key in empty_hists.keys(): 
        print(key)

        for spmode in spmodes:
            weight = 1
            if spmode=='-999':
                weight = .005
            else:
                weight = weights[spmode]

            # Divide these up by sp
            mask_sp= (ak_arr.spmode== spmode)
            
            # Cut on reasonable Lambda0, they might be duplicated in both SP and Data
            mask_fl = ak_arr[mask_sp]['Lambda0FlightLen']>=0

            # First order cuts ##################################################           
            # Make a flight-len cut on Lambda for B and Lambda0 variables
            if key[0]=='B' or key.find('Lambda0')==0:
                x = ak.flatten(ak_arr[mask_sp][key][mask_fl])
            else:
                x = ak_arr[mask_sp][key]
            
            empty_hists[key].fill(var=x, SP= spmode, cuts= f"0", weight= weight)

            # Second order cuts ##################################################
            mask_ntrk = (ak_arr['nTRK'] >= 5)

            mask_event = mask_sp & mask_ntrk
            
            # Cut on reasonable Lambda0, they might be duplicated in both SP and Data
            mask_fl = ak_arr[mask_event]['Lambda0FlightLen']>=0
            
            # Make a flight-len cut on Lambda for B and Lambda0 variables
            if key[0]=='B' or key.find('Lambda0')==0:
                x = ak.flatten(ak_arr[mask_event][key][mask_fl])
            else:
                x = ak_arr[mask_event][key]
            
            empty_hists[key].fill(var=x, SP= spmode, cuts= f"1", weight= weight)


            # Third order cuts ##################################################
            mask_event = mask_sp & mask_ntrk            
            mask_fl = ak_arr[mask_event]['Lambda0FlightLen']>=1

            # Make a flight-len cut on Lambda for B and Lambda0 variables
            if key[0]=='B' or key.find('Lambda0')==0:
                x = ak.flatten(ak_arr[mask_event][key][mask_fl])
            else:
                x = ak_arr[mask_event][key]
            
            empty_hists[key].fill(var=x, SP= spmode, cuts= f"2", weight= weight)

            ###################################################
            mask_event = mask_sp & mask_ntrk            
            mask_fl = ak_arr[mask_event]['Lambda0FlightLen']>=1

            mes = ak_arr[mask_event]['BpostFitMes']
            de  = ak_arr[mask_event]['BpostFitDeltaE']           
            
            # Stuff
            blinding_mask = (mes>5.27) & ((de>-.07) & (de<.07)) 
            fitarea_mask   =  (mes>5.2) & ((de>-.2) & (de<.2)) 

            var_mask = mask_fl & ~blinding_mask & fitarea_mask
            if spmode=='-999': # Don't blind signal
                var_mask = mask_fl & fitarea_mask

            # Make a flight-len cut on Lambda for B and Lambda0 variables
            if key[0]=='B' or key.find('Lambda0')==0:
                x = ak.flatten(ak_arr[mask_event][key][var_mask])
            else:
                x = ak_arr[mask_event][key]
            
            empty_hists[key].fill(var=x, SP= spmode, cuts= f"3", weight= weight)

    
    return "done"


In [ ]:
def fill_histograms_v2(ak_arr, empty_hists, spmodes=['998'], weights=[1.0]):
    ### Takes the dictionary of objects we made before and fills them 
    ### with the correct information, based on SP mode and Cut. 
    ### Each cut pares down the background and hopefully makes the signal more apparent

    # Save the result of the cuts in a dataframe
    df_dict = {}
    df_dict['var'] = []
    df_dict['cut'] = []
    df_dict['spmode'] = []
    df_dict['n'] = []

    for key in empty_hists.keys(): 
        print(key)

        for spmode in spmodes:
            #print(spmode)
            weight = 1
            if spmode=='-999':
                weight = .5
            else:
                weight = weights[spmode]

            all_cuts = {}
            mask_sp= (ak_arr.spmode== spmode)            
            mask_fl0 = (ak_arr['Lambda0FlightLen']>=0)
            mask_fl1 = (ak_arr['Lambda0FlightLen']>=1.25) & (ak_arr['Lambda0_unc_Mass']>=1.115683-0.003)& (ak_arr['Lambda0_unc_Mass']<=1.115683+0.003)
            mask_ntrk = (ak_arr['nTRK'] > 4)

            # Blinding
            mes = ak_arr['BpostFitMes']
            de  = ak_arr['BpostFitDeltaE']           
            # Stuff
            blinding_mask =   (mes>5.27) & ((de>-.07) & (de<.07)) 
            fitarea_mask   =  (mes>5.2) & ((de>-.2) & (de<.2)) 

            #var_mask = mask_fl & ~blinding_mask & fitarea_mask

            #print("blinding_mask lengths:", ak.num(blinding_mask, axis=1)[:10])
            #print("fitarea_mask lengths:", ak.num(fitarea_mask, axis=1)[:10])
            #print("mask_fl1 lengths:", ak.num(mask_fl1, axis=1)[:10])

            #print(ak.type(ak_arr['spmode']))
            #print(ak.type(ak_arr['nTRK']))
            #print(ak.type(ak_arr['Lambda0FlightLen']))
            #print(ak.type(ak_arr['BpostFitMes']))


            blinding_mask_flat= ak.any(blinding_mask, axis= 1)
            fitarea_mask_flat= ak.any(fitarea_mask, axis= 1)
            
            combined_mask_3= mask_sp & mask_ntrk & ~blinding_mask_flat & fitarea_mask_flat
            combined_mask_4= mask_sp & mask_ntrk & fitarea_mask_flat
            
            #print("mask_sp:", ak.type(mask_sp))
            #print("combined_mask_3:", ak.type(combined_mask_3))
            
            all_cuts['0'] = {'mask_ev': mask_sp, 'mask_part': mask_fl0}
            all_cuts['1'] = {'mask_ev': mask_sp & mask_ntrk, 'mask_part': mask_fl0}
            all_cuts['2'] = {'mask_ev': mask_sp & mask_ntrk, 'mask_part': mask_fl1}
            all_cuts['3']= {'mask_ev': combined_mask_3, 'mask_part': mask_fl1}
            all_cuts['4']= {'mask_ev': combined_mask_4, 'mask_part': mask_fl1}

            for cutname,cutmasks in all_cuts.items():

                #print(f"cutname: {cutname}")

                mask_ev=   cutmasks['mask_ev']
                mask_part= cutmasks['mask_part']

                n = -1
                # Apply the cuts and fill the histograms
                if key[0]=='B' or key.find('Lambda0')==0:
                    x = ak.flatten(ak_arr[mask_ev][key][mask_part[mask_ev]])
                else:
                    x = ak_arr[mask_ev][key]

                n = len(x)
                empty_hists[key].fill(var=x, SP= spmode, cuts= f"{cutname}", weight= weight)

                # Fill the dataframe dictionary
                df_dict['var'].append(key)
                df_dict['cut'].append(cutname)
                df_dict['spmode'].append(spmode)
                df_dict['n'].append(n)

    df = pd.DataFrame.from_dict(df_dict)
    
    return df


In [ ]:
all_hists = create_empty_histograms(hist_defs)

In [ ]:
#spmodes=['998', '1005', '1235', '1237', '-999']
bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

### Very important! If you redefine the SP modes at the bottom and run everything, you might get an error if they're different. Check your SP modes! 

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000

print(weights)
print()
print(spmodes)

In [ ]:
bkg_modes

In [ ]:
## SP HISTOGRAMS 
all_hists = create_empty_histograms(hist_defs)

#fill_histograms(ak_sp, all_hists, spmodes=spmodes, weights=weights);
df_cutflow_sp = fill_histograms_v2(ak_sp, all_hists, spmodes=spmodes, weights=weights);

In [ ]:
df_cutflow_sp

In [ ]:
mask = (df_cutflow_sp['var'] == 'BpostFitMes')
mask = mask & ((df_cutflow_sp['cut']=='0') | (df_cutflow_sp['cut']=='4'))
df_cutflow_sp[mask]

In [ ]:
mask_sp = (df_cutflow_sp['spmode'] == '-999')
mask_var = (df_cutflow_sp['var'] == 'BpostFitMes')

mask = mask_sp & mask_var & (df_cutflow_sp['cut']=='0')
norg = df_cutflow_sp[mask]['n'].values[0]

mask = mask_sp & mask_var & (df_cutflow_sp['cut']=='4')
ncut = df_cutflow_sp[mask]['n'].values[0]

print(f'{norg:8d} {ncut:8d}  {ncut/norg:.4f}')

In [ ]:
print(ak_sp['spmode'])


In [ ]:
###################################
# SP
###################################

#splist= np.unique(sp.to_list())
#print(splist)

dcuts= bat.get_final_masks(ak_sp, region_definitions= region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')


###################################
# Collision
###################################

dcuts_col= bat.get_final_masks(ak_data, region_definitions= region_definitions)




In [ ]:
## SP HISTOGRAMS 
all_hists = create_empty_histograms(hist_defs)

#fill_histograms(ak_sp, all_hists, spmodes=spmodes, weights=weights);
df_cutflow_sp = bat.fill_histograms_v3(ak_sp, empty_hists=all_hists, spmodes=spmodes, weights=weights, cuts=dcuts);

In [ ]:
#all_hists.keys()

In [ ]:
## DATA HISTOGRAMS 

#fill_histograms(ak_data, all_hists, spmodes=['0'], weights={'0':1})
#df_cutflow_data = fill_histograms_v2(ak_data, empty_hists=all_hists, spmodes=['0'], weights={'0':1})
df_cutflow_sp = bat.fill_histograms_v3(ak_data, empty_hists=all_hists, spmodes=['0'], weights={'0':1}, cuts=dcuts_col);

In [ ]:
mask = (df_cutflow_data['var'] == 'BpostFitMes') | (df_cutflow_data['var'] == 'BpostFitDeltaE')
#mask = mask & ((df_cutflow_data['cut']=='0') | (df_cutflow_data['cut']=='4'))
df_cutflow_data[mask]

In [ ]:
all_hists;

## Hist plotting

In [ ]:
def plot_histograms(all_hists, vars=[], bkg_spmodes=['998'], datamodes=['0'], sig_spmodes=['-999'], cut='0', save= True, overlay_data=True, only_stacked=False, fixed_grid=None):
    
    ### makes a directory (if it doesn't already exist) for these plots.
    ### plots will be saved to this dictionary if save= true
    current_dir= os.getcwd()
    directory = "BNV_pLambda_plots"
    path= os.path.join(current_dir,directory)
    if os.path.isdir(path)== False:
        os.mkdir(path)
    
    if len(vars) == 0:
        vars = list(all_hists.keys())

    ### color scheme dictionary
    cd= {}
    cd["998"]=  {"tab:blue"}
    cd["1005"]= {"tab:orange"}
    cd["-999"]= {"tab:brown"}
    cd["1235"]= {"tab:green"}
    cd["1237"]= {"tab:red"}
    cd["3981"]= {"tab:purple"}
    cd["3429"]= {"tab:pink"}
    cd["0"]= {"tab:cyan"}

    print(bkg_spmodes)

    if only_stacked and fixed_grid is not None:
        width = fixed_grid[0] * 5
        height = fixed_grid[1] * 3
        plt.figure(figsize=(width,height))           

    
    for axes_idx,var in enumerate(vars):
        
        h = all_hists[var]
                
        if only_stacked and not fixed_grid:   # if you only want the stacked sp and not the breakdown for individual modes 
            plt.figure(figsize=(5,3))
        elif only_stacked and fixed_grid:     # fixed grid is a grid of subplots 
            plt.subplot(fixed_grid[0], fixed_grid[1], axes_idx+1)
        else:
            plt.figure(figsize=(18,12))
            plt.subplot(3,3,1)
            
        h[:,bkg_spmodes,cut].stack('SP')[:].project('var').plot(stack=True, histtype="fill")
        h[:,sig_spmodes,cut].project('var').plot(histtype="step", color='red', label= "signal")

        if overlay_data:
            h[:,datamodes,cut].project('var').plot(histtype="errorbar", color='black', label='Data')

        plt.legend()
        plt.xlabel(plt.gca().get_xlabel(), fontsize=18)


        # If we are only plotting the stacked histograms, then we can skip over plotting them individually
        if not only_stacked:
            
            # Plot all the others 
            all_modes = bkg_spmodes + sig_spmodes + datamodes
            
            for idx,spmode in enumerate(all_modes):
                plt.subplot(3,3,idx+2)
                h[:,spmode,cut].project('var').plot(histtype="fill", label=spmode, color= cd[str(spmode)])
                plt.legend()
                plt.xlabel(plt.gca().get_xlabel(), fontsize=18)

        plt.tight_layout()
        
        if save== True:
            
            outfilename=f"plot_hist_cut{cut}_{var}.png" 
            if only_stacked and not fixed_grid:
                outfilename=f"plot_hist_cut{cut}_ONLY_STACKED_{var}.png" 
                
            plt.savefig(f"{path}/{outfilename}")

    if save== True and fixed_grid and only_stacked:
        # name of .png saved to computer based on fields specified on function call 
        varnames = "_".join(vars)
        if len(varnames)>24:
            varnames = 'TEST'
        outfilename=f"plot_hist_cut{cut}_ONLY_STACKED_FIXED_GRID_{varnames}.png" 
            
        plt.savefig(f"{path}/{outfilename}")



In [ ]:
print(bkg_spmodes)

print("_".join(bkg_spmodes))



In [ ]:
print(ak_sp['spmode'])


## Plot Area 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# FIX THIS - BELLIS
all_vars = list(all_hists.keys())

print(all_vars)
print(len(all_vars))

#'''
subset = ['BSphr', 'BpostFitDeltaE', 'BpostFitMes', 'BThrust', 'BCosThetaS', \
         'Lambda0_unc_Mass', 'R2', 'R2All', \
          'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
          'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
          'BThrustROE']
#'''
#subset = ['BSphr', 'BpostFitDeltaE', 'BpostFitMes', 'BThrust', 'BCosThetaS']

print(len(subset))
#plot_histograms(all_hists, vars=['Lambda0_unc_Mass'], bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='2');
#plot_histograms(all_hists, vars=['BpostFitMes'], bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='-1');
plot_histograms(all_hists, vars=['BpostFitMes'], bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='-1', overlay_data=True, only_stacked=True,);

#plot_histograms(all_hists, vars=['BpostFitDeltaE'], bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='1');
#plot_histograms(all_hists, vars=['BpostFitDeltaE'], bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='0', overlay_data=False);
#plot_histograms(all_hists, vars=all_vars, bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='2');

#plot_histograms(all_hists, vars=["Lambda0_unc_Mass"], bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='0', overlay_data=True, only_stacked=True, save= False);

#plot_histograms(all_hists, vars=["BpostFitMes", "BpostFitDeltaE", "Lambda0_unc_Mass", "BCosSphr"], bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut='3', overlay_data=True, only_stacked=True, save= True, fixed_grid=(2,2));
#plot_histograms(all_hists, vars=subset, bkg_spmodes=['998', '1005'], sig_spmodes=sig_spmodes, datamodes=['0'], cut='3', overlay_data=True, only_stacked=True, save= True, fixed_grid=(4,5));


## Blinding and fitting region

In [ ]:
mask = ak_sp['Lambda0FlightLen']>=0

sigmask = ak_sp['spmode'] == '-999'


mes = ak.flatten(ak_sp['BpostFitMes'][mask & sigmask])
de  = ak.flatten(ak_sp['BpostFitDeltaE'][mask & sigmask])


plt.figure(figsize=(12,12))
plt.subplot(3,2,1)
plt.plot(mes, de, '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

norg = len(mes)
print(f"# original: {norg}")

# Stuff
blinding_mask = (mes>5.27) & ((de>-.07) & (de<.07)) 
fitarea_mask   =  (mes>5.2) & ((de>-.2) & (de<.2)) 

plt.subplot(3,2,3)
plt.plot(mes[blinding_mask], de[blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,4)
plt.plot(mes[~blinding_mask], de[~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")


plt.subplot(3,2,5)
plt.plot(mes[fitarea_mask], de[fitarea_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,6)
plt.plot(mes[fitarea_mask & ~blinding_mask], de[fitarea_mask & ~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")


In [ ]:
print(ak_sp['spmode'])


In [ ]:
mask = ak_sp['Lambda0FlightLen']>=0

sigmask = ak_sp['spmode'] == '-999'


mes = ak.flatten(ak_sp['BpostFitMes'][mask & sigmask])
de  = ak.flatten(ak_sp['BpostFitDeltaE'][mask & sigmask])


plt.figure(figsize=(12,12))
plt.subplot(3,2,1)
plt.plot(mes, de, '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

norg = len(mes)
print(f"# original: {norg}")

# Stuff

blinding_mask = (mes>5.27) & ((de>-.07) & (de<.07)) 
fitarea_mask   =  (mes>5.2) & ((de>-.2) & (de<.2)) 

plt.subplot(3,2,3)
plt.plot(mes[blinding_mask], de[blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,4)
plt.plot(mes[~blinding_mask], de[~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")


plt.subplot(3,2,5)
plt.plot(mes[fitarea_mask], de[fitarea_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,6)
plt.plot(mes[fitarea_mask & ~blinding_mask], de[fitarea_mask & ~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")


## Sandbox

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(1,2,1)
x = ak_sp['nTRK']
plt.hist(x, bins=20, range=(0,20))


plt.subplot(1,2,2)
x = ak_data['nTRK']
plt.hist(x,bins=20, range=(0,20))

;

In [ ]:
x = 10

print(x)

x = 20

print(x)

In [ ]:
x = 10

print(x)

y = x

print(y)

x = 20

print(x,y)

In [ ]:
x = [10]

print(x)

y = x

print(y)

x[0] = 20

print(x,y)

In [ ]:
x = np.array([10])

print(x)

y = x

print(y)

x[0] = 20

print(x,y)



y[0] = 999

print(x,y)

In [ ]:
# Make a copy of the actual values and give it a new name!
x = [10]

print(x)

y = list(x)
d1idx = ak_sp['Lambda0d1Idx']

print(y)

x[0] = 20

print(x,y)

In [ ]:
# Make a new array!
x = np.array([10])

print(x)

y = x.copy()

print(y)

x[0] = 20

print(x,y)

In [ ]:
h = all_hists['BSphr']

h[:,['998'],:].project('var').plot(histtype="errorbar", color='black')

In [ ]:
all_hists;

In [ ]:
h = Hist.new.Reg(100, 0, 20, name="Lambda0postFitFlight", label=r"") \
         .StrCat([], name="SP", label="SP modes", growth=True)\
         .StrCat([], name="cuts", label="Cuts", growth=True)\
         .Weight()

#for key,value in ak_sp.items():
#    x = value['BpostFitMes'][:,0]
#    h.fill(BpostFitMes=x, SP=key, cuts=f"0", weight=1)

spmodes = ['998','1005', '1235', '1237', '3981']
for spmode in spmodes:
    mask = ak_sp.spmode == spmode
    x = ak_sp[mask]['Lambda0postFitFlight'][:,0]
    h.fill(Lambda0postFitFlight=x, SP=spmode, cuts=f"0", weight=1)

In [ ]:
h.stack('SP')[:].project('Lambda0postFitFlight').plot(stack=True, histtype="fill")

plt.legend()

In [ ]:
plt.figure(figsize=(18,12))

spmodes = ['998','1005', '1235', '1237', '3981']
for idx,spmode in enumerate(spmodes):
    plt.subplot(2,3,idx+1)
    h[:,spmode,:].project('Lambda0postFitFlight').plot(histtype="fill", label=spmode)
    plt.legend()

In [ ]:

j = Hist.new.Reg(100, 5.2, 5.3, name="BpostFitMes", label=r"M$_{ES}$ [GeV/c$^2$]") \
         .StrCat([], name="SP", label="SP modes", growth=True)\
         .StrCat([], name="cuts", label="Cuts", growth=True)\
         .Weight()


bkg_spmodes = ['998','1005', '1235', '1237', '991']#, '3429']
spmodes = bkg_spmodes + ['-999']
#spmodes = ['991']

for spmode in spmodes:
    mask = ak_sp.spmode == spmode
    x = ak_sp[mask]['BpostFitMes'][:,0]

    weight = 1
    if spmode == '-999':
        weight = 0.02
    
    j.fill(BpostFitMes=x, SP=spmode, cuts=f"0", weight=weight)

In [ ]:
j[:,:,:].stack('SP')[:].project('BpostFitMes').plot(stack=True, histtype="fill")
j[:,['-999'],:].project('BpostFitMes').plot(histtype="errorbar", color='black', label= "data")

plt.legend()

In [ ]:
j[:,bkg_spmodes,:].stack('SP')[:].project('BpostFitMes').plot(stack=True, histtype="fill")
j[:,['-999'],:].project('BpostFitMes').plot(histtype="errorbar", color='black')

plt.legend()

In [ ]:
mask = ak_data['Lambda0FlightLen'] >=0
x = ak.flatten(ak_data['Lambda0_unc_Mass'][mask])

plt.hist(x,bins=100);

print(ak_data['Lambda0_unc_Mass'][mask])

# Trying out PID selector cuts

https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/PID/PID_Selector_List

In [ ]:
# Get these maps first
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")

print(pips.selectors)
print()
print(pps.selectors)

In [ ]:
# Let's try this for SP (Monte Carlo)

def PID_masks(ak_arr, \
              lamp_selector='VeryLooseKMProtonSelection', \
              lampi_selector='SuperLooseKMPionMicroSelection', \
              Bp_selector='VeryLooseKMProtonSelection', \
             verbosity=0):
    
    # Proton and pion information from the Lambda decay
    # These are the index of the proton (d1) and pion (d2) in those lists
    d1idx = ak_arr['Lambda0d1Idx']
    d2idx = ak_arr['Lambda0d2Idx']
    
    d1lund = ak_arr['Lambda0d1Lund']
    d2lund = ak_arr['Lambda0d2Lund']
    
    Bd2idx = ak_arr['Bd2Idx']
    Bd2lund = ak_arr['Bd2Lund']

    if verbosity==1:
        print(d1lund)
        print(d2lund)
        print(Bd2lund)
        print()
        
        print(d1idx)
        print(d2idx)
        print(Bd2idx)
        print()
    
    trkidx_proton = ak_sp['pTrkIdx']
    trk_selector_map_proton = ak_sp['pSelectorsMap']
    
    trkidx_pion = ak_sp['piTrkIdx']
    trk_selector_map_pion = ak_sp['piSelectorsMap']
    
    # Proton
    pbits = bat.calculate_bits_for_PID_selector(trkidx_proton, trk_selector_map_proton, verbose=verbosity)
    # Pion
    pibits = bat.calculate_bits_for_PID_selector(trkidx_pion, trk_selector_map_pion, verbose=verbosity)
    
    
    #selector_proton = 'TightKMProtonSelection'
    #selector_pion = 'TightKMPionMicroSelection'
    #print(f"Now trying to create a mask with {selector_proton}")
    #print(f"Now trying to create a mask with {selector_pion}")
    
    
    mask_bool_proton = bat.mask_PID_selection(pbits[d1idx], lamp_selector, pps)
    mask_bool_protonB = bat.mask_PID_selection(pbits[Bd2idx], Bp_selector, pps)
        
    mask_bool_pion = bat.mask_PID_selection(pibits[d2idx], lampi_selector, pips)

    return mask_bool_proton, mask_bool_pion, mask_bool_protonB


mask_bool_proton, mask_bool_pion, mask_bool_protonB = PID_masks(ak_sp, \
              lamp_selector='VeryLooseKMProtonSelection', \
              lampi_selector='SuperLooseKMPionMicroSelection', \
              Bp_selector='SuperTightKMProtonSelection', \
              verbosity=0)


# Make some plots

spmask = (ak_sp['spmode']=='998')

lamfl_mask = (ak_sp['Lambda0FlightLen']>1)

lammass = ak_sp['Lambda0_unc_Mass']
mes = ak_sp['BpostFitMes']

mask_pid =      mask_bool_proton & mask_bool_pion & mask_bool_protonB
mask_particle = mask_pid[spmask] & lamfl_mask[spmask] 

plt.figure(figsize=(12,5));

plt.subplot(1,2,1)
plt.hist(ak.flatten(lammass[spmask & lamfl_mask]),bins=100)
plt.hist(ak.flatten(lammass[spmask & lamfl_mask & mask_pid]),bins=100)

print(len(ak.flatten(lammass[spmask & lamfl_mask])))
print(len(ak.flatten(lammass[spmask & lamfl_mask & mask_pid])))


plt.subplot(1,2,2)
plt.hist(ak.flatten(mes[spmask & lamfl_mask]),bins=100, range=(5.2,5.3))
plt.hist(ak.flatten(mes[spmask & lamfl_mask & mask_pid]),bins=100, range=(5.2,5.3))

print(len(ak.flatten(mes[spmask & lamfl_mask])))
print(len(ak.flatten(mes[spmask & lamfl_mask & mask_pid])))

;

In [ ]:
sp_info = {}

sp_info["1235"]= {"Process":"B+B-","# generated":694106000, "# in LambdaVeryVeryLoose Skim":43724348,"Cross section [nb]":.525, "Scaling Factor (to 430 fb^{-1})":.3192}
sp_info["1237"]= {"Process":"B0B0bar","# generated":717995000, "# in LambdaVeryVeryLoose Skim":48917772,"Cross section [nb]":.525, "Scaling Factor (to 430 fb^{-1})":.3149}
sp_info["998"]= {"Process":"uubar, ddbar, ssbar","# generated":1595892000, "# in LambdaVeryVeryLoose Skim":83956877,"Cross section [nb]":2.09, "Scaling Factor (to 430 fb^{-1})":.2507}
sp_info["1005"]= {"Process":"ccbar","# generated":1128544000, "# in LambdaVeryVeryLoose Skim":67408513,"Cross section [nb]":1.3, "Scaling Factor (to 430 fb^{-1})":.4962}
sp_info["2400"]= {"Process":"e+e-","# generated":0, "# in LambdaVeryVeryLoose Skim":0,"Cross section [nb]":40, "Scaling Factor (to 430 fb^{-1})":36.4742}
sp_info["3429"]= {"Process":"tau+tau-","# generated":55595000, "# in LambdaVeryVeryLoose Skim":1591920,"Cross section [nb]":.919, "Scaling Factor (to 430 fb^{-1})":.2447}
sp_info["3981"]= {"Process":"mu+mu-gamma","# generated":506776000, "# in LambdaVeryVeryLoose Skim":36121,"Cross section [nb]":1.147, "Scaling Factor (to 430 fb^{-1})":.7951}


info_df= pd.DataFrame.from_dict(sp_info, orient= "index")


In [ ]:
ak_sp.fields

In [ ]:
d1idx = ak_sp['Lambda0d1Idx']
d2idx = ak_sp['Lambda0d2Idx']

pro_cos= ak_sp["pcosth"][d1idx]
pi_cos= ak_sp["picosth"][d2idx]

pro_mom= ak_sp["pp3"][d1idx]
pi_mom= ak_sp["pip3"][d2idx]

plt.plot(ak.flatten(pro_cos), ak.flatten(pro_mom), "g.", alpha= .01)
#plt.hist(ak.flatten(pro_cos), bins= 100)

In [ ]:
import hist
from hist import Hist

In [ ]:
h= hist.NamedHist(
    hist.axis.Regular(
        50,-1,1, name= "Cos", label= "cos($\Theta$)", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,0,6, name= "Mom", label= "momentum [GeV/c]", underflow= False, overflow= False),)

h.fill(Cos= ak.flatten(pro_cos) ,Mom= ak.flatten(pro_mom))


In [ ]:
h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

In [ ]:
h= hist.NamedHist(
    hist.axis.Regular(
        50,-1,1, name= "Cos", label= "cos($\Theta$)", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,0,6, name= "Mom", label= "momentum [GeV/c]", underflow= False, overflow= False),)

h.fill(Cos= ak.flatten(pi_cos) ,Mom= ak.flatten(pi_mom))


In [ ]:
h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

In [ ]:
from analysis_variables import *
region_definitions

In [ ]:
d1idx = ak_sp['Lambda0d1Idx']
d2idx = ak_sp['Lambda0d2Idx']

pro_cos= ak_sp["pcosth"][d1idx]
pi_cos= ak_sp["picosth"][d2idx]

pro_mom= ak_sp["pp3"][d1idx]
pi_mom= ak_sp["pip3"][d2idx]

sig_mask= ak_sp["spmode"]=="-999"

#plt.plot(ak.flatten(pro_cos), ak.flatten(pro_mom), "g.", alpha= .01)
#plt.hist(ak.flatten(pro_cos), bins= 100)

In [ ]:
sig_pi_cos= ak.flatten(pi_cos[sig_mask])
sig_pi_mom= ak.flatten(pi_mom[sig_mask])

MC_pi_cos= ak.flatten(pi_cos[~sig_mask])
MC_pi_mom= ak.flatten(pi_mom[~sig_mask])


sig_pro_cos= ak.flatten(pro_cos[sig_mask])
sig_pro_mom= ak.flatten(pro_mom[sig_mask])

MC_pro_cos= ak.flatten(pro_cos[~sig_mask])
MC_pro_mom= ak.flatten(pro_mom[~sig_mask])

In [ ]:
plt.plot(sig_pi_cos, sig_pi_mom,"b.", alpha= 0.05)
plt.ylim(0,2)
plt.figure()
plt.plot(MC_pi_cos, MC_pi_mom,"b.", alpha= 0.05)
plt.ylim(0,2)

### Next two are plots of 1) signal MC protons and 2) background MC protons 

In [ ]:
h= hist.NamedHist(
    hist.axis.Regular(
        50,-1,1, name= "Cos", label= "cos($\Theta$)", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,0,6, name= "Mom", label= "momentum [GeV/c]", underflow= False, overflow= False),)

h.fill(Cos= sig_pro_cos ,Mom= sig_pro_mom)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
plt.savefig(f'BNV_pLambda_plots/plot_lambda0proton_sig_p_vs_costh.png')

In [ ]:
h= hist.NamedHist(
    hist.axis.Regular(
        50,-1,1, name= "Cos", label= "cos($\Theta$)", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,0,6, name= "Mom", label= "momentum [GeV/c]", underflow= False, overflow= False),)

h.fill(Cos= MC_pro_cos ,Mom= MC_pro_mom)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
plt.savefig(f'BNV_pLambda_plots/plot_lambda0proton_bkg_p_vs_costh.png')

### Plots of 1) signal MC pions from the lambda and 2) background MC pions from the lambda

In [ ]:
h= hist.NamedHist(
    hist.axis.Regular(
        50,-1,1, name= "Cos", label= "cos($\Theta$)", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,0,1.5, name= "Mom", label= "momentum [GeV/c]", underflow= False, overflow= False),)

h.fill(Cos= sig_pi_cos ,Mom= sig_pi_mom)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
plt.savefig(f'BNV_pLambda_plots/plot_lambda0pion_sig_p_vs_costh.png')

In [ ]:
h= hist.NamedHist(
    hist.axis.Regular(
        50,-1,1, name= "Cos", label= "cos($\Theta$)", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,0,1.5, name= "Mom", label= "momentum [GeV/c]", underflow= False, overflow= False),)

h.fill(Cos= MC_pi_cos ,Mom= MC_pi_mom)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
plt.savefig(f'BNV_pLambda_plots/plot_lambda0pion_bkg_p_vs_costh.png')

# Signal and fitting area masks

In [ ]:
def get_signal_mask(data, region_definitions):
    mes= data["BpostFitMes"]
    de= data["BpostFitDeltaE"]
    signal_mask = (mes > region_definitions["signal MES"][0]) & ((de>region_definitions["signal DeltaE"][0]) & (de<region_definitions["signal DeltaE"][1])) 
    #fitarea_mask   =  (mes>5.2) & ((de>-.2) & (de<.2))
    return signal_mask 

spmask = ak_sp['spmode']=='998'

print(spmask)

mes= ak_sp[spmask]["BpostFitMes"]
de= ak_sp[spmask]["BpostFitDeltaE"]

print(mes)

sigmask = get_signal_mask(ak_sp, region_definitions)


mes= ak_sp[spmask]["BpostFitMes"][~sigmask[spmask]]
de= ak_sp[spmask]["BpostFitDeltaE"][~sigmask[spmask]]

print(mes)

x = ak.flatten(mes)
y = ak.flatten(de)

plt.plot(x,y,'.',markersize=0.5)
plt.xlim(5.2,5.3)
plt.ylim(-0.3, 0.3)
    

In [ ]:
def get_fit_mask(data, region_definitions):
    mes= data["BpostFitMes"]
    de= data["BpostFitDeltaE"]
    fit_mask = (mes > region_definitions["fitting MES"][0]) & ((de>region_definitions["fitting DeltaE"][0]) & (de<region_definitions["fitting DeltaE"][1])) 
    return fit_mask 


spmask = ak_sp['spmode']=='998'

print(spmask)

mes= ak_sp[spmask]["BpostFitMes"]
de= ak_sp[spmask]["BpostFitDeltaE"]

print(mes)

fitmask = get_fit_mask(ak_sp, region_definitions)


mes= ak_sp[spmask]["BpostFitMes"][fitmask[spmask]]
de= ak_sp[spmask]["BpostFitDeltaE"][fitmask[spmask]]

print(mes)

x = ak.flatten(mes)
y = ak.flatten(de)

plt.plot(x,y,'.',markersize=0.5)
plt.xlim(5.2,5.3)
plt.ylim(-0.3, 0.3)
    

In [ ]:
sig_FL= ak.flatten(ak_sp["Lambda0FlightLen"][sigmask])
sig_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][sigmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,11, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= sig_FL ,Mass= sig_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
plt.savefig(f'BNV_pLambda_plots/plot_sig_mass_vs_FL.png')

In [ ]:
bkg_FL= ak.flatten(ak_sp["Lambda0FlightLen"][~sigmask])
bkg_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][~sigmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,11, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= bkg_FL ,Mass= bkg_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)

plt.savefig(f'BNV_pLambda_plots/plot_bkg_mass_vs_FL.png')

## Sideband masks (WIP)

In [ ]:
from analysis_variables import *
region_definitions

In [ ]:
#s1_DEu= ak_sp["BpostFitDeltaE"] >= region_definitions["sideband 1 DeltaE"][0] 
#s1_DEl= ak_sp["BpostFitDeltaE"] <= region_definitions["sideband 1 DeltaE"][1]
#sideband_MES= ak_sp["BpostFitMes"] >= region_definitions["sideband MES"][0]

#sideband_mask= ak_sp[s1_DEu & s1_DEl]

#ak_sp["Lambda0_unc_Mass"][sideband1_MES][both_DE_sideband_mask]

In [ ]:
sig_FL= ak.flatten(ak_sp["Lambda0FlightLen"][sigmask])
sig_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][sigmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,30, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= sig_FL ,Mass= sig_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
plt.savefig(f'BNV_pLambda_plots/plot_sig_mass_vs_FL_large_scale.png')

In [ ]:
sig_FL= ak.flatten(ak_sp["Lambda0FlightLen"][sigmask])
sig_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][sigmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,2, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= sig_FL ,Mass= sig_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)

plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
plt.savefig(f'BNV_pLambda_plots/plot_sig_mass_vs_FL_short_scale.png')

In [ ]:
plt.hist(bkg_mass, bins=100);

## Background Stuff

In [ ]:
bkg_FL= ak.flatten(ak_sp["Lambda0FlightLen"][~sigmask])
bkg_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][~sigmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,2, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= bkg_FL ,Mass= bkg_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)

plt.savefig(f'BNV_pLambda_plots/plot_bkg_mass_vs_FL_short_scale.png')

In [ ]:
bkg_FL= ak.flatten(ak_sp["Lambda0FlightLen"][fitmask])
bkg_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][fitmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,30, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= bkg_FL ,Mass= bkg_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
#plt.xscale("log")
plt.savefig(f'BNV_pLambda_plots/plot_bkg_mass_vs_FL_large_scale.png')

## Sideband stuff 

In [ ]:
side_FL= ak.flatten(ak_sp["Lambda0FlightLen"][fitmask])
side_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][fitmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,30, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= side_FL ,Mass= side_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
#plt.xscale("log")
plt.savefig(f'BNV_pLambda_plots/plot_side_mass_vs_FL_large_scale.png')

In [ ]:
side_FL= ak.flatten(ak_sp["Lambda0FlightLen"][fitmask])
side_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][fitmask])


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,2, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= side_FL ,Mass= side_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)
#plt.xscale("log")
plt.savefig(f'BNV_pLambda_plots/plot_side_mass_vs_FL_short_scale.png')

# Bellis edit to test cut

In [ ]:
#spmode_to_test = '-999'
#spmode_to_test = '998'
spmode_to_test = '1005'


spmask = (ak_sp['spmode']==spmode_to_test) & (ak_sp['Lambda0FlightLen']>=0)

flmask = ak_sp['Lambda0FlightLen']>0.20
lammassmask = (ak_sp['Lambda0_unc_Mass']<1.118) & (ak_sp['Lambda0_unc_Mass']>1.113)

mask0 = spmask
mask1 = spmask & flmask & lammassmask

bkg_FL= ak.flatten(ak_sp["Lambda0FlightLen"][mask0])
bkg_mass= ak.flatten(ak_sp["Lambda0_unc_Mass"][mask0])

bkg_FL_tight= ak.flatten(ak_sp["Lambda0FlightLen"][mask1])
bkg_mass_tight= ak.flatten(ak_sp["Lambda0_unc_Mass"][mask1])

print(len(bkg_FL))
print(len(bkg_FL_tight))


In [ ]:
h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,2, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= bkg_FL ,Mass= bkg_mass)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)

plt.savefig(f'BNV_pLambda_plots/plot_bkg_mass_vs_FL_short_scale.png')
;


In [ ]:


h= hist.NamedHist(
    hist.axis.Regular(
        50,-0.1,2, name= "FL", label= "distance [cm]", underflow= False, overflow= False), 
    hist.axis.Regular(
        50,1.1,1.13, name= "Mass", label= "mass [$GeV/c^2$]", underflow= False, overflow= False),)

h.fill(FL= bkg_FL_tight ,Mass= bkg_mass_tight)

h.plot2d_full(
    main_cmap= "plasma",
    top_ls= "--",
    top_color= "orange",
    top_lw= 2,
    side_ls= ":",
    side_lw= 2,
    side_color= "steelblue"
)


plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
plt.ylabel(plt.gca().get_ylabel(), fontsize=18)

plt.savefig(f'BNV_pLambda_plots/plot_bkg_mass_vs_FL_short_scale.png')
;
